In [47]:
import numpy as np
import os
import re

import multiprocess as mp
import warnings
warnings.filterwarnings("ignore")

import plotly.graph_objs as go

from trProcess import *

In [48]:
def WriteCdf (fileName, data):
    data.sort ()
    curIndex = 0
    if (len (data) > 10000):
        stepSize = len (data) / 10000
    else:
        stepSize = 1
    with open (fileName, 'w') as f:
        lastData = -1
        while curIndex < len (data):
            if data[int (curIndex)] != lastData:
                f.write ("%d\t%.4f\n" % (data[int (curIndex)], curIndex / len (data)))
                lastData = data[int (curIndex)]
            curIndex += stepSize

In [49]:
def ProcessBaseline (exps, expconf, tracenum, indir, baselinelist):
    apps = expconf.split("_")
    tot_delay_dur, tot_plt, tot_max_delay, tot_all_delay = {}, {}, {}, {}
    for name in baselinelist.keys():
        tot_delay_dur[name] = []
        tot_plt[name] = []
        tot_max_delay[name] = []
        tot_all_delay[name] = []
    for exp in exps:
        if not re.search (expconf, exp):
            continue
        cur_delay_dur, cur_plt, cur_max_delay, cur_all_delay = {}, {}, {}, {}
        for name, baseline in baselinelist.items():
            cur_all_delay[name] = []
            p90FctTrace, maxAppATrace, maxAppBTrace, jfiTrace = [], [], [], []
            for traceid in range(tracenum):
                trace = "maxwell_20.0_0.01_300_%d.pitree-trace" % traceid
                conf = os.path.join (indir, exp, baseline, trace)
                    
                curAvgFct, curP90Fct, fctLastStart, fctFirstEnd, fcts = fctTraceProcessor(os.path.join (conf, 'fct.tr'))
                p90FctTrace.append(curP90Fct)
                
                bws = []
                duration = max(fcts) + 500
                curMaxApp, curAppDur, _, curAllDelay = appTraceProcessor (os.path.join (conf, 'app.tr'), startTime=14000 + 13000 * apps[0].count("f"), duration=duration)
                curMaxAppFlows = 0
                for port, appDelay in curAppDur.items():
                    if port in range(10000, 20000):
                        curMaxAppFlows = max(curMaxAppFlows, appDelay)
                maxAppATrace.append(curMaxAppFlows)
                maxAppBTrace.append(max(curMaxApp.values()))

                bws = np.array(bws)
                jfiTrace.append(pow(np.nansum(bws),2) / (bws.size * np.nansum(np.square(bws))))
                cur_all_delay[name].extend(list(curAllDelay.values())[0])
            cur_plt[name] = np.average(p90FctTrace)
            cur_delay_dur[name] = np.average(maxAppATrace)
            cur_max_delay[name] = np.average(maxAppBTrace)
            
        
        for name in baselinelist.keys():
            tot_delay_dur[name].append(cur_delay_dur[name])
            tot_max_delay[name].append(cur_max_delay[name])
            tot_all_delay[name].extend(cur_all_delay[name])
            tot_plt[name].append(cur_plt[name] - cur_plt['FQ'])

    for name in baselinelist.keys():
        WriteCdf(name + "-delay-dur.cdf", tot_delay_dur[name])
        WriteCdf(name + "-max-delay.cdf", tot_max_delay[name])
        WriteCdf(name + "-all-delay.cdf", tot_all_delay[name])
        WriteCdf(name + "-plt.cdf", tot_plt[name])

    

In [50]:
tracenum = 1
baselinelist = {
    "FQ": "FqCoDel_0_500ms1000", 
    "FIFO": "PfifoFast_0_1000",
    # "HHF": "Hhf_0_1000",
    # "SJF": "StrictPriority_90_1000", 
    # "CoDel": "CoDel_0_5ms1000",
    # "RED": "Red_0_1000",
    "Confucius": "Auto_0_1.00_ExpClass4.00",
    "CBQ (1:1)": "Dwrr_9_1.00",
    # "CBQ (1:5)": "Dwrr_9_0.20",
    # "DualQ" : "DualQ_9_1000",
    # "StrictPriority" : "StrictPriority_9_1000",
}

expconfs = [
    # "^4fff0_Video1_Web.*000$",
    # "^45_Video1_Web.*000$",
    "^46_Video1_Web.*000$",
    # "^49_Video1_Web.*000$",
]
indir = "ns-3.34/logs/"
exps = os.listdir (indir)
for expconf in expconfs:
    ProcessBaseline(exps, expconf, tracenum, indir, baselinelist)
